<a href="https://colab.research.google.com/github/castrokelly/MBA/blob/main/Processamento%20Massivo%20em%20Paralelo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <span style="color:blue">MBA em Ciência de Dados</span>
# <span style="color:blue">Análise de Dados com Base em Processamento Massivo em Paralelo</span>

## <span style="color:blue">Avaliação Final - Notebook </span>

**Material Produzido por:**<br>
>**Profa. Dra. Cristina Dutra de Aguiar**<br>


**CEMEAI - ICMC/USP São Carlos**

Este *notebook* deve conter as respostas para as consultas analíticas solicitadas nas Questões 7, 8, 9 e 10. É possível especificar as consultas analíticas usando Pandas, o método spark.sql() ou os métodos do módulo pyspark.sql. As consultas devem ser especificadas na seção 6, na célula indicada para resposta.

**IMPORTANTE**

- **As respostas para as Questões 7, 8, 9 e 10 somente serão consideradas se forem especificadas no *notebook*. Portanto, independentemente da alternativa estar certa ou errada, se a consulta analítica correspondente não for especificada, a alternativa será considerada errada.**

- **Caso uma alternativa esteja certa, porém a especificação da consulta analítica correspondente estiver errada no *notebook*, a alternativa será considerada errada.**


O *notebook* contém a constelação de fatos da BI Solutions que deve ser utilizada para responder às questões e também todas as bibliotecas, bases de dados, inicializações, instalações, importações, geração de dataFrames, geração de visões temporárias e conversão dos tipos de dados necessárias para a realização da questão. Portanto, o *notebook* está preparado para ser executado usando Pandas, o método spark.sql() e os métodos do módulo pyspark.sql.

O uso do *framework* Spark requer diversas configurações no ambiente de desenvolvimento para executar o *notebook*. Dado que tal complexidade foge do escopo de nossa disciplina, recomenda-se que o *notebook* seja executado na plataforma de desenvolvimento COLAB. O uso do COLAB  proporciona um ambiente de desenvolvimento pré-configurado e remove a complexidade de instalação e configuração de pacotes e *frameworks* que são utilizados na disciplina.

**INSTRUÇÕES DE ENTREGA**

**O que deve ser entregue:**
- **O notebook com as respostas no formato .ipynb**
- **O notebook com as respostas no formato .pdf**

**Ambos arquivos devem ser nomeados usando o primeiro nome e o último sobrenome do aluno. Por exemplo: CristinaAguiar.ipynb e CristinaAguiar.pdf.**

Boa avaliação!

#1 Constelação de Fatos da BI Solutions

A aplicação de *data warehousing* da BI Solutions utiliza como base uma constelação de fatos, conforme descrita a seguir.

**Tabelas de dimensão**

- data (dataPK, dataCompleta, dataDia, dataMes, dataBimestre, dataTrimestre, dataSemestre, dataAno)
- funcionario (funcPK, funcMatricula, funcNome, funcSexo, funcDataNascimento, funcDiaNascimento, funcMesNascimento, funcAnoNascimento, funcCidade, funcEstadoNome, funcEstadoSigla, funcRegiaoNome, funcRegiaoSigla, funcPaisNome, funcPaisSigla)
- equipe (equipePK, equipeNome, filialNome, filialCidade, filialEstadoNome, filialEstadoSigla, filialRegiaoNome, filialRegiaoSigla, filialPaisNome, filialPaisSigla)
- cargo (cargoPK, cargoNome, cargoRegimeTrabalho, cargoEscolaridadeMinima, cargoNivel)
- cliente (clientePK, clienteNomeFantasia, clienteSetor, clienteCidade, clienteEstadoNome, clienteEstadoSigla, clienteRegiaoNome, clienteRegiaoSigla, clientePaisNome, clientePaisSigla)

**Tabelas de fatos**
- pagamento (dataPK, funcPK, equipePK, cargoPK, salario, quantidadeLancamentos)
- negociacao (dataPK, equipePK, clientePK, receita, quantidadeNegociacoes)


#2 Obtenção dos Dados da BI Solutions


## 2.1 Baixando o Módulo wget

Para baixar os dados referentes ao esquema relacional da constelação de fatos da BI Solutions, é utilizado o módulo  **wget**. O comando a seguir realiza a instalação desse módulo. <br>

In [1]:
#instalando o módulo wget
%%capture
!pip install -q wget
!mkdir data

## 2.2 Obtenção dos Dados das Tabelas de Dimensão

Os comandos a seguir baixam os dados que povoam as tabelas de dimensão.

In [2]:
#baixando os dados das tabelas de dimensão
import wget

url = "https://raw.githubusercontent.com/GuiMuzziUSP/Data_Mart_BI_Solutions/main/data.csv"
wget.download(url, "data/data.csv")

url = "https://raw.githubusercontent.com/GuiMuzziUSP/Data_Mart_BI_Solutions/main/funcionario.csv"
wget.download(url, "data/funcionario.csv")

url = "https://raw.githubusercontent.com/GuiMuzziUSP/Data_Mart_BI_Solutions/main/equipe.csv"
wget.download(url, "data/equipe.csv")

url = "https://raw.githubusercontent.com/GuiMuzziUSP/Data_Mart_BI_Solutions/main/cargo.csv"
wget.download(url, "data/cargo.csv")

url = "https://raw.githubusercontent.com/GuiMuzziUSP/Data_Mart_BI_Solutions/main/cliente.csv"
wget.download(url, "data/cliente.csv")

'data/cliente.csv'

## 2.3 Obtenção dos Dados Tabelas de Fatos

Os comandos a seguir baixam os dados que povoam as tabelas de fatos.

In [3]:
#baixando os dados das tabelas de fatos
url = "https://raw.githubusercontent.com/GuiMuzziUSP/Data_Mart_BI_Solutions/main/pagamento.csv"
wget.download(url, "data/pagamento.csv")

url = "https://raw.githubusercontent.com/GuiMuzziUSP/Data_Mart_BI_Solutions/main/negociacao.csv"
wget.download(url, "data/negociacao.csv")

'data/negociacao.csv'

# 3 Apache Spark Cluster

## 3.1 Instalação

Neste *notebook* é criado um *cluster* Spark composto apenas por um **nó mestre**. Ou seja, o *cluster* não possui um ou mais **nós de trabalho** e o **gerenciador de cluster**. Nessa configuração, as tarefas (*tasks*) são realizadas no próprio *driver* localizado no **nó mestre**.

Para que o cluster possa ser criado, primeiramente é instalado o Java Runtime Environment (JRE) versão 8.

In [4]:
#instalando Java Runtime Environment (JRE) versão 8
%%capture
!apt-get remove openjdk*
!apt-get update --fix-missing
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

Na sequência, é feito o *download* do Apache Spark versão 3.0.0.

In [5]:
#baixando Apache Spark versão 3.0.0
%%capture
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop2.7.tgz
!tar xf spark-3.0.0-bin-hadoop2.7.tgz && rm spark-3.0.0-bin-hadoop2.7.tgz

Na sequência, são configuradas as variáveis de ambiente JAVA_HOME e SPARK_HOME. Isto permite que tanto o Java quanto o Spark possam ser encontrados.

In [6]:
import os
#configurando a variável de ambiente JAVA_HOME
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
#configurando a variável de ambiente SPARK_HOME
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop2.7"

Por fim, são instalados dois pacotes da linguagem de programação Python, cujas funcionalidades são descritas a seguir.

> **Pacote findspark:** Usado para ler a variável de ambiente SPARK_HOME e armazenar seu valor na variável dinâmica de ambiente PYTHONPATH. Como resultado, Python pode encontrar a instalação do Spark.

> **Pacote pyspark:** PySpark é a API do Python para Spark. Ela possibilita o uso de Python, considerando que o *framework* Apache Spark encontra-se desenvolvido na linguagem de programação Scala.

In [7]:
%%capture
#instalando o pacote findspark
!pip install -q findspark==1.4.2
#instalando o pacote pyspark
!pip install -q pyspark==3.0.0

## 3.2 Conexão

PySpark não é adicionado ao *sys.path* por padrão. Isso significa que não é possível importá-lo, pois o interpretador da linguagem Python não sabe onde encontrá-lo.

Para resolver esse aspecto, é necessário instalar o módulo `findspark`. Esse módulo mostra onde PySpark está localizado. Os comandos a seguir têm essa finalidade.


In [8]:
#importando o módulo findspark
import findspark
#carregando a variávels SPARK_HOME na variável dinâmica PYTHONPATH
findspark.init()

Depois de configurados os pacotes e módulos e inicializadas as variáveis de ambiente, é possível iniciar o uso do Spark na aplicação de `data warehousing`. Para tanto, é necessário importar o comando `SparkSession` do módulo `pyspark.sql`. São utilizados os seguintes conceitos: <br>

- `SparkSession`: permite a criação de `DataFrames`. Como resultado, as tabelas relacionais podem ser manipuladas por meio de `DataFrames` e é possível realizar consultas OLAP por meio de comandos SQL. <br>
- `builder`: cria uma instância de SparkSession. <br>
- `appName`: define um nome para a aplicação, o qual pode ser visto na interface de usuário web do Spark. <br>
- `master`: define onde está o nó mestre do *cluster*. Como a aplicação é executada localmente e não em um *cluster*, indica-se isso pela *string* `local` seguida do parâmetro `[*]`. Ou seja, define-se que apenas núcleos locais são utilizados.
- `getOrCreate`: cria uma SparkSession. Caso ela já exista, retorna a instância existente.


**Observação**: A lista completa de todos os parâmetros que podem ser utilizados na inicialização do *cluster* pode ser encontrada neste [link](https://spark.apache.org/docs/latest/spark-standalone.html#cluster-launch-scripts).

In [9]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("pyspark-notebook").master("local[*]").getOrCreate()

# 4 Geração dos DataFrames em Pandas da BI Solutions

Nesta seção são gerados os DataFrames em Pandas. Atenção aos nomes desses DataFrames.


In [10]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [11]:
cargoPandas = pd.read_csv('https://raw.githubusercontent.com/GuiMuzziUSP/Data_Mart_BI_Solutions/main/cargo.csv')
clientePandas = pd.read_csv('https://raw.githubusercontent.com/GuiMuzziUSP/Data_Mart_BI_Solutions/main/cliente.csv')
dataPandas = pd.read_csv('https://raw.githubusercontent.com/GuiMuzziUSP/Data_Mart_BI_Solutions/main/data.csv')
equipePandas = pd.read_csv('https://raw.githubusercontent.com/GuiMuzziUSP/Data_Mart_BI_Solutions/main/equipe.csv')
funcionarioPandas = pd.read_csv('https://raw.githubusercontent.com/GuiMuzziUSP/Data_Mart_BI_Solutions/main/funcionario.csv')
negociacaoPandas = pd.read_csv('https://raw.githubusercontent.com/GuiMuzziUSP/Data_Mart_BI_Solutions/main/negociacao.csv')
pagamentoPandas = pd.read_csv('https://raw.githubusercontent.com/GuiMuzziUSP/Data_Mart_BI_Solutions/main/pagamento.csv')

# 5 Geração dos DataFrames em Spark da BI Solutions

Nesta seção são gerados dos DataFrames em Spark. Atenção aos nomes desses DataFrames.


## 5.1 Criação dos DataFrames

In [12]:
#criando os DataFrames em Spark
cargo = spark.read.csv(path="data/cargo.csv", header=True, sep=",")
cliente = spark.read.csv(path="data/cliente.csv", header=True, sep=",")
data = spark.read.csv(path="data/data.csv", header=True, sep=",")
equipe = spark.read.csv(path="data/equipe.csv", header=True, sep=",")
funcionario = spark.read.csv(path="data/funcionario.csv", header=True, sep=",")
negociacao = spark.read.csv(path="data/negociacao.csv", header=True, sep=",")
pagamento = spark.read.csv(path="data/pagamento.csv", header=True, sep=",")

## 5.2 Atualização dos Tipos de Dados

Nos comandos a seguir, primeiro são identificados quais colunas de quais `DataFrames` devem ser do tipo de dado inteiro. Na sequência, ocorre a conversão. Por fim, são exibidos os esquemas dos `DataFrames`, possibilitando visualizar a mudança de tipo de dados das colunas especificadas.

In [13]:
# identificando quais colunas de quais DataFrames devem ser do tipo de dado inteiro
colunas_cargo = ["cargoPK"]
colunas_cliente = ["clientePK"]
colunas_data = ["dataPK", "dataDia", "dataMes", "dataBimestre", "dataTrimestre", "dataSemestre", "dataAno"]
colunas_equipe = ["equipePK"]
colunas_funcionario = ["funcPK", "funcDiaNascimento", "funcMesNascimento", "funcAnoNascimento"]
colunas_negociacao = ["equipePK", "clientePK", "dataPK", "quantidadeNegociacoes"]
colunas_pagamento = ["funcPK", "equipePK", "dataPK", "cargoPK", "quantidadeLancamentos"]

In [14]:
# importando o tipo de dado desejado
from pyspark.sql.types import IntegerType


# atualizando o tipo de dado das colunas especificadas
# substituindo as colunas já existentes

for coluna in colunas_cargo:
  cargo = cargo.withColumn(coluna, cargo[coluna].cast(IntegerType()))

for coluna in colunas_cliente:
  cliente = cliente.withColumn(coluna, cliente[coluna].cast(IntegerType()))

for coluna in colunas_data:
  data = data.withColumn(coluna, data[coluna].cast(IntegerType()))

for coluna in colunas_equipe:
  equipe = equipe.withColumn(coluna, equipe[coluna].cast(IntegerType()))

for coluna in colunas_funcionario:
  funcionario = funcionario.withColumn(coluna, funcionario[coluna].cast(IntegerType()))

for coluna in colunas_negociacao:
  negociacao = negociacao.withColumn(coluna, negociacao[coluna].cast(IntegerType()))

for coluna in colunas_pagamento:
  pagamento = pagamento.withColumn(coluna, pagamento[coluna].cast(IntegerType()))

Nos comandos a seguir, primeiro são identificados quais colunas de quais `DataFrames` devem ser do tipo de dado número de ponto flutuante. Na sequência, ocorre a conversão. Por fim, são exibidos os esquemas dos `DataFrames`, possibilitando visualizar a mudança de tipo de dados das colunas especificadas.

In [15]:
# identificando quais colunas de quais DataFrames devem ser do tipo de dado número de ponto flutuante
colunas_negociacao = ["receita"]
colunas_pagamento = ["salario"]

In [16]:
# importando o tipo de dado desejado
from pyspark.sql.types import FloatType


# atualizando o tipo de dado das colunas especificadas
# substituindo as colunas já existentes

for coluna in colunas_negociacao:
  negociacao = negociacao.withColumn(coluna, negociacao[coluna].cast(FloatType()))

for coluna in colunas_pagamento:
  pagamento = pagamento.withColumn(coluna, pagamento[coluna].cast(FloatType()))

In [17]:
# importando funções adicionais
from pyspark.sql.functions import round, desc

## 5.3 Criação de Visões Temporárias

In [18]:
#criando as visões temporárias
cargo.createOrReplaceTempView("cargo")
cliente.createOrReplaceTempView("cliente")
data.createOrReplaceTempView("data")
equipe.createOrReplaceTempView("equipe")
funcionario.createOrReplaceTempView("funcionario")
negociacao.createOrReplaceTempView("negociacao")
pagamento.createOrReplaceTempView("pagamento")

# 6 Respostas

Lembre-se que é possível especificar as consultas analíticas usando Pandas, o método spark.sql() ou os métodos do módulo pyspark.sql.

## Especificação da Consulta Analítica da Questão 7

In [65]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, round, col, desc

# Inicializar SparkSession
spark = SparkSession.builder.appName("Questao7").getOrCreate()

resultado = spark.sql("""
    SELECT
        f.funcSexo AS SEXO,            -- Seleciona o campo funcSexo como SEXO
        f.funcAnoNascimento AS ANO,   -- Seleciona o campo funcAnoNascimento como ANO
        ROUND(AVG(p.salario), 2) AS MEDIASALARIO  -- Calcula a média dos salários e arredonda para 2 casas decimais
    FROM
        pagamento p                   -- Tabela de fatos "pagamento"
    JOIN
        funcionario f                 -- Junta com a tabela de dimensão "funcionario"
    ON
        p.funcPK = f.funcPK           -- Junta as tabelas pela chave primária do funcionário
    WHERE
        f.funcRegiaoNome = 'NORDESTE' -- Filtra apenas funcionários que residem na região NORDESTE
    GROUP BY
        CUBE(f.funcSexo, f.funcAnoNascimento)  -- Gera todas as combinações de agregações
    ORDER BY
        SEXO ASC NULLS LAST,          -- Ordena por SEXO, com NULLs no final
        ANO ASC NULLS LAST,           -- Ordena por ANO, com NULLs no final
        MEDIASALARIO ASC              -- Ordena por média salarial em ordem crescente
""")

# Exibir os resultados
resultado.show(50)


+----+----+------------+
|SEXO| ANO|MEDIASALARIO|
+----+----+------------+
|   F|1953|    11842.55|
|   F|1990|     5935.93|
|   F|null|     7043.42|
|   M|1951|     4116.83|
|   M|1952|     3635.55|
|   M|1965|     1919.34|
|   M|1966|    14995.02|
|   M|1967|     6686.45|
|   M|1978|     2581.24|
|   M|1979|     6300.52|
|   M|1980|     2085.85|
|   M|1990|     8742.07|
|   M|null|     7183.82|
|null|1951|     4116.83|
|null|1952|     3635.55|
|null|1953|    11842.55|
|null|1965|     1919.34|
|null|1966|    14995.02|
|null|1967|     6686.45|
|null|1978|     2581.24|
|null|1979|     6300.52|
|null|1980|     2085.85|
|null|1990|     7782.07|
|null|null|     7151.73|
+----+----+------------+



In [64]:
# Média salarial geral
media_geral = resultado.filter("Sexo IS NULL AND Ano IS NULL")
media_geral.show()

+----+----+------------+
|SEXO| ANO|MEDIASALARIO|
+----+----+------------+
|null|null|     7151.73|
+----+----+------------+



In [55]:
# Maior média salarial (sexo M, ano 1966)
maior_media = resultado.filter("Sexo = 'M' AND Ano = 1966")
maior_media.show()

+----+----+------------+
|Sexo| Ano|MEDIASALARIO|
+----+----+------------+
|   M|1966|    14995.02|
+----+----+------------+



In [56]:
# Menor média salarial (sexo M, ano 1965)
menor_media = resultado.filter("Sexo = 'M' AND Ano = 1965")
menor_media.show()

+----+----+------------+
|Sexo| Ano|MEDIASALARIO|
+----+----+------------+
|   M|1965|     1919.34|
+----+----+------------+



In [57]:
# Média salarial das funcionárias de todos os anos (quando ANO é nulo)
media_funcionarias = resultado.filter("Sexo = 'F' AND Ano IS NULL")
media_funcionarias.show()

+----+----+------------+
|Sexo| Ano|MEDIASALARIO|
+----+----+------------+
|   F|null|     7043.42|
+----+----+------------+



In [58]:
# Média salarial dos funcionários masculinos de todos os anos (quando ANO é nulo)
media_funcionarios = resultado.filter("Sexo = 'M' AND Ano IS NULL")
media_funcionarios.show()

+----+----+------------+
|Sexo| Ano|MEDIASALARIO|
+----+----+------------+
|   M|null|     7183.82|
+----+----+------------+



In [59]:
# Média salarial para o ano de 1990, independentemente do sexo
media_1990 = resultado.filter("Ano = 1990")
media_1990.show()

+----+----+------------+
|Sexo| Ano|MEDIASALARIO|
+----+----+------------+
|null|1990|     7782.07|
|   F|1990|     5935.93|
|   M|1990|     8742.07|
+----+----+------------+



Consulta Analítica Realizada:

A consulta calculou as médias salariais dos funcionários que residem na região Nordeste, agrupando por `sexo` e `ano de nascimento`.

Valores nulos foram incluídos no agrupamento para contemplar casos onde sexo ou ano de nascimento não estão disponíveis.

A média salarial foi arredondada para duas casas decimais.

* Resultados Obtidos:

Foram retornadas 24 linhas.

Os anos de nascimento variam de 1951 a 1990.

Existem valores nulos tanto no campo sexo quanto no campo ano de nascimento.

## Especificação da Consulta Analítica da Questão 8

In [83]:
# Listar as colunas disponíveis na visão temporária `cliente`
print("Colunas disponíveis na visão cliente:")
spark.sql("SELECT * FROM cliente").printSchema()


Colunas disponíveis na visão cliente:
root
 |-- clientePK: integer (nullable = true)
 |-- clienteNomeFantasia: string (nullable = true)
 |-- clienteSetor: string (nullable = true)
 |-- clienteCidade: string (nullable = true)
 |-- clienteEstadoNome: string (nullable = true)
 |-- clienteEstadoSigla: string (nullable = true)
 |-- clienteRegiaoNome: string (nullable = true)
 |-- clienteRegiaoSigla: string (nullable = true)
 |-- clientePaisNome: string (nullable = true)
 |-- clientePaisSigla: string (nullable = true)



In [86]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum as _sum, round

# Inicializar SparkSession
spark = SparkSession.builder.appName("Questao8").getOrCreate()

# 1. Consulta intermediária
consulta_intermediaria = spark.sql("""
    SELECT
        d.dataAno AS ANO,                      -- Seleciona o ano da data
        cl.clienteSetor AS SETOR,              -- Seleciona o setor do cliente
        SUM(n.receita) AS SOMARECEITA          -- Soma a receita
    FROM
        negociacao n
    JOIN
        cliente cl ON cl.clientePK = n.clientePK -- Junção com a tabela cliente
    JOIN
        data d ON n.dataPK = d.dataPK          -- Junção com a tabela data
    WHERE
        d.dataAno IN (2019, 2020)              -- Filtra os anos de interesse
        AND cl.clienteEstadoNome = 'SAO PAULO' -- Filtra clientes localizados no Estado de São Paulo
    GROUP BY
        d.dataAno, cl.clienteSetor             -- Agrupa por ano e setor
    ORDER BY
        d.dataAno ASC, cl.clienteSetor ASC     -- Ordena pelo ano e setor
""")

# Print da consulta intermediária
print("Consulta intermediária após WHERE e GROUP BY:")
consulta_intermediaria.show()


Consulta intermediária após WHERE e GROUP BY:
+----+-------------------+------------------+
| ANO|              SETOR|       SOMARECEITA|
+----+-------------------+------------------+
|2019|BEBIDAS E ALIMENTOS|        3995904.25|
|2019|            CREDITO|1347436.2541503906|
|2019|              SAUDE|     1914114.15625|
|2019|         TECNOLOGIA| 5778946.271240234|
|2019|          VESTUARIO|1680762.3587646484|
|2020|BEBIDAS E ALIMENTOS| 4264757.815917969|
|2020|            CREDITO|1393427.5520019531|
|2020|              SAUDE| 1292370.146118164|
|2020|         TECNOLOGIA| 3567224.553955078|
|2020|          VESTUARIO|2782399.5897216797|
+----+-------------------+------------------+



Consulta Intermediária:

Os dados foram filtrados inicialmente pelos anos de 2019 e 2020 (`d.dataAno IN (2019, 2020)`) e pelos clientes localizados no estado de São Paulo (`cl.clienteEstadoNome = 'SAO PAULO'`).

Após aplicar os filtros, a soma das receitas por ano e setor foi calculada usando a função `SUM(n.receita)` e agrupada por `ANO` e `SETOR`.

O resultado da consulta intermediária incluiu as colunas `ANO`, `SETOR`, e `SOMARECEITA`.

In [87]:
# Aplicação do filtro (HAVING) separadamente
resultado_final = (
    consulta_intermediaria
    .filter(col("SOMARECEITA") > 3900000)  # Filtro de receita superior a R$ 3.900.000
    .select(
        col("ANO"),
        col("SETOR"),
        round(col("SOMARECEITA"), 2).alias("TOTALRECEITA")  # Arredondamento para duas casas decimais
    )
    .orderBy("ANO", "SETOR")  # Ordenação final
)

# Print do resultado final
print("Resultado final após aplicação do filtro HAVING:")
resultado_final.show()

Resultado final após aplicação do filtro HAVING:
+----+-------------------+------------+
| ANO|              SETOR|TOTALRECEITA|
+----+-------------------+------------+
|2019|BEBIDAS E ALIMENTOS|  3995904.25|
|2019|         TECNOLOGIA|  5778946.27|
|2020|BEBIDAS E ALIMENTOS|  4264757.82|
+----+-------------------+------------+



Aplicação do Filtro (`HAVING`):

Após a consulta intermediária, foi aplicado o filtro para considerar apenas os setores com receitas superiores a R$ 3.900.000,00 `(SOMARECEITA > 3900000`).
O resultado final foi refinado para exibir as colunas `ANO`, `SETOR`, e `TOTALRECEITA`, onde a soma das receitas foi arredondada para duas casas decimais.

Ordenação dos Resultados:

O resultado final foi ordenado pelas colunas `ANO` e `SETOR`.

Resultados:

3 linhas

Para o ano de 2019, os setores de maior receita foram BEBIDAS E ALIMENTOS e TECNOLOGIA, com receitas de `R$ 3.959.904,25` e `R$ 5.778.946,27`, respectivamente.

Para o ano de 2020, o setor de maior receita foi BEBIDAS E ALIMENTOS, com R$ 4.264.757,82.

## Especificação da Consulta Analítica da Questão 9

In [90]:
# Listar as colunas disponíveis na visão temporária 'equipe'
print("Colunas disponíveis na visão equipe:")
spark.sql("SELECT * FROM equipe").printSchema()


Colunas disponíveis na visão equipe:
root
 |-- equipePK: integer (nullable = true)
 |-- equipeNome: string (nullable = true)
 |-- filialNome: string (nullable = true)
 |-- filialCidade: string (nullable = true)
 |-- filialEstadoNome: string (nullable = true)
 |-- filialEstadoSigla: string (nullable = true)
 |-- filialRegiaoNome: string (nullable = true)
 |-- filialRegiaoSigla: string (nullable = true)
 |-- filialPaisNome: string (nullable = true)
 |-- filialPaisSigla: string (nullable = true)



In [92]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum as _sum, round, when

# Inicializar SparkSession
spark = SparkSession.builder.appName("Questao9").getOrCreate()

# Consulta intermediária: calcular receitas por equipe e ano
consulta_receitas = spark.sql("""
    SELECT
        e.equipeNome AS EQUIPENOME,               -- Nome da equipe
        e.filialNome AS EQUIPEFILIAL,             -- Nome da filial da equipe (corrigido)
        d.dataAno AS ANO,                         -- Ano
        SUM(n.receita) AS TOTALRECEITA            -- Soma das receitas
    FROM
        negociacao n
    JOIN
        equipe e ON n.equipePK = e.equipePK       -- Junção com a tabela equipe
    JOIN
        data d ON n.dataPK = d.dataPK             -- Junção com a tabela data
    WHERE
        d.dataAno IN (2019, 2020)                 -- Filtrar anos de 2019 e 2020
    GROUP BY
        e.equipeNome, e.filialNome, d.dataAno     -- Agrupar por equipe, filial, e ano
    ORDER BY
        e.equipeNome, e.filialNome, d.dataAno     -- Ordenar para facilitar visualização
""")

# Mostrar a consulta intermediária
print("Consulta de receita por equipe e ano:")
consulta_receitas.show()

Consulta de receita por equipe e ano:
+--------------+--------------------+----+--------------------+
|    EQUIPENOME|        EQUIPEFILIAL| ANO|        TOTALRECEITA|
+--------------+--------------------+----+--------------------+
| APP - DESKTOP|RIO DE JANEIRO - ...|2019|  2417619.8994140625|
| APP - DESKTOP|RIO DE JANEIRO - ...|2020|  2290441.2983398438|
| APP - DESKTOP|SAO PAULO - AV. P...|2019|  2009714.0009765625|
| APP - DESKTOP|SAO PAULO - AV. P...|2020|  2146181.2392578125|
|  APP - MOBILE|CAMPO GRANDE - CE...|2019|  1280914.8969726562|
|  APP - MOBILE|CAMPO GRANDE - CE...|2020|  1347929.7026367188|
|  APP - MOBILE|RIO DE JANEIRO - ...|2019|   1645371.005859375|
|  APP - MOBILE|RIO DE JANEIRO - ...|2020|  1289304.9975585938|
|  APP - MOBILE|SAO PAULO - AV. P...|2019|  1779690.5947265625|
|  APP - MOBILE|SAO PAULO - AV. P...|2020|  1243670.5493164062|
|BI & ANALYTICS|     RECIFE - CENTRO|2019|1.2310646513671875E7|
|BI & ANALYTICS|     RECIFE - CENTRO|2020|    8791572.87109375|
|B

In [98]:
# Criar a visão temporária 'consulta_receitas'
consulta_receitas.createOrReplaceTempView("consulta_receitas")

In [100]:
# Calcular receitas de 2019, 2020, e a diferença entre os dois anos
resultado_final = spark.sql("""
    SELECT
        r2019.EQUIPENOME,
        r2019.EQUIPEFILIAL,
        COALESCE(r2019.TOTALRECEITA, 0) AS RECEITA2019,
        COALESCE(r2020.TOTALRECEITA, 0) AS RECEITA2020,
        COALESCE(r2020.TOTALRECEITA, 0) - COALESCE(r2019.TOTALRECEITA, 0) AS DIFERENCA
    FROM
        (SELECT EQUIPENOME, EQUIPEFILIAL, TOTALRECEITA
         FROM consulta_receitas WHERE ANO = 2019) r2019
    FULL OUTER JOIN
        (SELECT EQUIPENOME, EQUIPEFILIAL, TOTALRECEITA
         FROM consulta_receitas WHERE ANO = 2020) r2020
    ON
        r2019.EQUIPENOME = r2020.EQUIPENOME AND r2019.EQUIPEFILIAL = r2020.EQUIPEFILIAL
    ORDER BY
        DIFERENCA DESC, EQUIPENOME, EQUIPEFILIAL
""")

# Mostrar o resultado final
print("Resultado final:")
resultado_final.show()


Resultado final:
+--------------+--------------------+--------------------+------------------+-------------------+
|    EQUIPENOME|        EQUIPEFILIAL|         RECEITA2019|       RECEITA2020|          DIFERENCA|
+--------------+--------------------+--------------------+------------------+-------------------+
| APP - DESKTOP|SAO PAULO - AV. P...|  2009714.0009765625|2146181.2392578125|    136467.23828125|
|           WEB|SAO PAULO - AV. P...|   647854.0006103516| 751983.7419433594| 104129.74133300781|
|  APP - MOBILE|CAMPO GRANDE - CE...|  1280914.8969726562|1347929.7026367188|   67014.8056640625|
|           WEB|CAMPO GRANDE - CE...|   956287.2945556641| 1017644.055480957|  61356.76092529297|
| APP - DESKTOP|RIO DE JANEIRO - ...|  2417619.8994140625|2290441.2983398438|-127178.60107421875|
|  APP - MOBILE|RIO DE JANEIRO - ...|   1645371.005859375|1289304.9975585938|-356066.00830078125|
|           WEB|RIO DE JANEIRO - ...|  1037993.7048339844| 612673.8998413086| -425319.8049926758|
|  

Com base nos dados obtidos:

* A equipe APP - DESKTOP da filial SAO PAULO - AV. PAULISTA apresentou uma diferença positiva de 13.647,23, indicando que a receita de 2020 foi maior que a de 2019.

* A equipe WEB da filial SAO PAULO - AV. PAULISTA teve uma diferença positiva de 104.219,13.

* A equipe BI & ANALYTICS da filial RECIFE - CENTRO apresentou a maior diferença negativa, com -351.917,62, indicando que sua receita caiu drasticamente de 2019 para 2020.

* As equipes APP - MOBILE e WEB em várias filiais também apresentaram receitas com diferenças variadas.

## Especificação da Consulta Analítica da Questão 10

In [102]:
print("Esquema da visão temporária 'funcionario':")
spark.sql("SELECT * FROM funcionario").printSchema()


Esquema da visão temporária 'funcionario':
root
 |-- funcPK: integer (nullable = true)
 |-- funcMatricula: string (nullable = true)
 |-- funcNome: string (nullable = true)
 |-- funcSexo: string (nullable = true)
 |-- funcDataNascimento: string (nullable = true)
 |-- funcDiaNascimento: integer (nullable = true)
 |-- funcMesNascimento: integer (nullable = true)
 |-- funcAnoNascimento: integer (nullable = true)
 |-- funcCidade: string (nullable = true)
 |-- funcEstadoNome: string (nullable = true)
 |-- funcEstadoSigla: string (nullable = true)
 |-- funcRegiaoNome: string (nullable = true)
 |-- funcRegiaoSigla: string (nullable = true)
 |-- funcPaisNome: string (nullable = true)
 |-- funcPaisSigla: string (nullable = true)



In [105]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum as _sum, round

# Inicializar SparkSession
spark = SparkSession.builder.appName("Questao10").getOrCreate()

# Consulta para calcular os gastos em salários
consulta_salarios = spark.sql("""
    SELECT
        f.funcEstadoNome AS ESTADO,             -- Nome do estado
        f.funcRegiaoNome AS REGIAO,             -- Nome da região
        ROUND(SUM(p.salario), 2) AS TOTALSALARIOS -- Soma dos salários
    FROM
        pagamento p
    JOIN
        funcionario f ON p.funcPK = f.funcPK   -- Junção com a tabela de funcionários
    JOIN
        cargo c ON p.cargoPK = c.cargoPK       -- Junção com a tabela de cargos # Changed the join condition
    JOIN
        data d ON p.dataPK = d.dataPK          -- Junção com a tabela de datas
    WHERE
        d.dataAno = 2020                       -- Filtrar apenas o ano de 2020
        AND c.cargoNivel = 'SENIOR'            -- Filtrar apenas cargos SENIOR
    GROUP BY
        f.funcEstadoNome, f.funcRegiaoNome     -- Agrupar por estado e região
    ORDER BY
        f.funcRegiaoNome, f.funcEstadoNome     -- Ordenar por região e estado
""")

# Mostrar o resultado
print("Consulta de salários por estado e região:")
consulta_salarios.show()


Consulta de salários por estado e região:
+--------------+--------+-------------+
|        ESTADO|  REGIAO|TOTALSALARIOS|
+--------------+--------+-------------+
|    PERNAMBUCO|NORDESTE|    822273.25|
|  MINAS GERAIS| SUDESTE|   2133676.23|
|RIO DE JANEIRO| SUDESTE|   1094470.34|
|     SAO PAULO| SUDESTE|   4623213.74|
|        PARANA|     SUL|   2006090.29|
+--------------+--------+-------------+



Resultados Obtidos:

A consulta retornou os seguintes valores para os gastos totais com salários no ano de `2020`, considerando apenas funcionários com nível de cargo `SENIOR`:

* PERNAMBUCO (NORDESTE): 822.273,25
* MINAS GERAIS (SUDESTE): 1.333.766,23
* RIO DE JANEIRO (SUDESTE): 1.049.476,34
* SÃO PAULO (SUDESTE): 4.623.213,74
* PARANÁ (SUL): 2.008.699,29